In [17]:
import glob
import os
from csio.graph_load import ArcanGraphLoader
project = "avro"
in_path = "../data/arcanOutput"


filepath = glob.glob(os.path.join(in_path, project, "dep-graph-*.graphml"))[0]

In [18]:
graph = ArcanGraphLoader().load(filepath)

In [19]:
import numpy

def load_embeddings(path):
    embeddings = {}
    with open(path, "rt", encoding="utf8") as inf:
        for line in inf:
            splitLines = line.split()
            word = splitLines[0]
            embedding = numpy.array([float(value) for value in splitLines[1:]])
            embeddings[word] = embedding

    return embeddings

In [20]:
embedding_method = "fastText"
embeddings_path = f"../data/embeddings/{embedding_method}/{project}.vec"

embeddings = load_embeddings(embeddings_path)

In [21]:
def dict_to_array(graph, embeddings):
    doc_emb = []
    for node in graph.vs:
        name = node["filePathReal"]
        if name == ".":
            continue

        node_features = numpy.array(embeddings[name]).astype(numpy.float)
        doc_emb.append(node_features)

    return doc_emb

In [22]:
from sklearn.cluster import DBSCAN, KMeans
from sklearn.decomposition import PCA

X = numpy.array(dict_to_array(graph, embeddings))
#pca = PCA(n_components=20)
#X_red = pca.fit_transform(X)

#z = DBSCAN(min_samples=4).fit_predict(X_red)


In [23]:
partitions = []
for i in range(7, 13):
    clustering = KMeans(n_clusters=10).fit(X)
    partitions.append(clustering.labels_)

In [24]:
from cdlib.algorithms import eva
import networkx as nx
import random
#l1 = ['A', 'B', 'C', 'D']
l2 = ["E", "F", "G"]
#g_attr = nx.barabasi_albert_graph(100, 5)

for i, part in enumerate(partitions):
    A = graph.get_edgelist()
    g_attr = nx.Graph(A)
    labels=dict()
    for node in g_attr.nodes():
        labels[node]={"l1": random.choice(l2)}#, "l2":random.choice(l2)}
    communities = eva(g_attr, labels, alpha=0.8)
    print(i, len(communities.communities))

KeyError: 'G'